In [43]:
import sqlite3
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

# sqlitebrowser

# create diagram https://dbdiagram.io/d

# Create sql from csv

In [2]:
genstudio = pd.read_csv('./data/genstudio.csv', dtype={'Position':str})
metadata = pd.read_csv('./data/metadata.csv')

#print(genstudio.shape, metadata.shape)

In [14]:
def get_scheme_df(df):
    '''
    Create SQL scheme from DataFrame colnames.
    Return SQL scheme and list with processed colnames.
    '''
    col_names, types = df.columns, df.dtypes
    scheme_df = ['row_id INTEGER PRIMARY KEY']
    new_names = ['row_id']


    for name, contain in zip(col_names[1:], types[1:]):
        name = name.replace("-", "").replace("  ", " ").replace(" ", "_")
        new_names.append(name)

        if 'int' in str(contain):
            scheme_df.append(' '.join([name, 'INTEGER']))
        
        elif 'float' in str(contain):
            scheme_df.append(' '.join([name, 'REAL']))
        
        else:
            scheme_df.append(' '.join([name, 'TEXT']))
            
    scheme_df = ', '.join(scheme_df)

    return scheme_df, new_names

In [16]:
scheme_genstudio, new_names = get_scheme_df(genstudio)
genstudio.columns = new_names

scheme_metadata, new_names = get_scheme_df(metadata)
metadata.columns = new_names

In [78]:
def bid_DB(genstudio, metadata, scheme_genstudio, scheme_metadata, path='./data/bid_DB.db'):
    
    connection = sqlite3.connect(path)

    connection.execute('''CREATE TABLE IF NOT EXISTS SNP_data (
    Sample_ID TEXT, 
    SNP_Name TEXT, 
    SNP TEXT, 
    Position TEXT, 
    Chr TEXT,
    sex TEXT)''')

    connection.execute(f'''CREATE TABLE IF NOT EXISTS metadata ({scheme_metadata},
                        FOREIGN KEY (sex) REFERENCES SNP_data (sex))''')

    connection.execute(f'''CREATE TABLE IF NOT EXISTS genstudio ({scheme_genstudio},
    FOREIGN KEY (Sample_ID) REFERENCES metadata (dna_chip_id),
    FOREIGN KEY (Sample_ID) REFERENCES metadata (Sample_ID),
    FOREIGN KEY (SNP_Name) REFERENCES metadata (SNP_Name),
    FOREIGN KEY (SNP) REFERENCES metadata (SNP),
    FOREIGN KEY (Position) REFERENCES metadata (Position),
    FOREIGN KEY (Chr) REFERENCES metadata (Chr))''')

    connection.commit()

    genstudio.to_sql('genstudio', connection, if_exists='append', index=False)
    metadata.to_sql('metadata', connection, if_exists='append', index=False)

    connection.close()

In [79]:
path='./data/bid_DB.db'
bid_DB(genstudio, metadata, scheme_genstudio, scheme_metadata, path)

# Selection

In [80]:
connection = sqlite3.connect('./data/bid_DB.db')

query = '''SELECT metadata.dna_chip_id, genstudio.SNP_Name, genstudio.SNP, genstudio.Position, genstudio.Chr, 
           metadata.sex
           FROM genstudio, metadata
           WHERE metadata.dna_chip_id = genstudio.Sample_ID AND genstudio.SNP = '[A/G]'
'''

res = connection.execute(query)

print(len(res.fetchall()))

connection.close()

## Add selection to DB

In [82]:
connection = sqlite3.connect('./data/bid_DB.db')

query = '''INSERT INTO SNP_data (Sample_ID, SNP_Name, SNP, Position, Chr, sex)
SELECT genstudio.Sample_ID, genstudio.SNP_Name, genstudio.SNP, genstudio.Position, genstudio.Chr, 
metadata.sex 
FROM genstudio, metadata 
WHERE metadata.dna_chip_id = genstudio.Sample_ID'''

connection.execute(query)
connection.commit()
connection.close()

# Create and add from web

In [205]:
def get_genes():
    
    GENE_URL = 'https://genomics.senescence.info/genes/allgenes.php'
    
    resp = requests.get(GENE_URL)
    soup = BeautifulSoup(resp.content, 'lxml')
    
    genes_list = []
    
    table = soup.find("table", {"class":"results-table"} )
    rows = table.find_all('tr')

    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        
        if cols:    
            genes_list.append(cols[0])
    
    return genes_list


def snp_search(gene_name):
    '''
    Search first 20 SNP in ncbi snp database. Return soup object.
    '''
    SNP_URL = 'https://www.ncbi.nlm.nih.gov/snp/'
    query_str = {'term': gene_name + '[Gene Name]'}
    
    resp = requests.get(SNP_URL, params=query_str)
    soup = BeautifulSoup(resp.content, 'lxml')
    
    return soup


def parse_soup(soup):
    '''
    Extract SNP ID from soup.
    Return list with rs-id.
    '''
    rs_id = []
    
    for result in soup.find_all('div', {'class': 'rprt'}):
        rs_id.append(result.find('div', {'class': 'supp'}).find('a').text)
        
    return rs_id


def get_id_list(genes_list):
    '''
    Search first 20 SNP in ncbi snp database and extract SNP ID from soup.
    Wait 5 seconds between get requests.
    Return lists with SNP ID`s for all genes.
    '''
    id_list = []
    
    for gene_name in genes_list:
        #time.sleep(5)
        
        soup = snp_search(str(gene_name))
        rs_id = parse_soup(soup)

        if rs_id:
            id_list.append(rs_id)
            
    return id_list


In [216]:
def add_to_sql(snp_name, position, chromosome, alleles, db_build, assembly_build,
               clin_sig, n_publications):
    '''
    Open connection to SNP_human DB, add some info to it and close connection.
    Return nothing.
    '''
    connection = sqlite3.connect('./data/SNP_human.db')
    
    query_1 = '''INSERT INTO SNP_data (SNP_Name, Alleles, Position, Chr, dbSNP_Build, Assembly_Build)
                                       VALUES (?, ?, ?, ?, ?, ?)'''
    
    connection.execute(query_1, [snp_name, alleles, position, chromosome, db_build, assembly_build])
    
    query_2 = '''INSERT INTO clin_SNP (SNP_Name, Clinical_Significance, N_Publications)
                                       VALUES (?, ?, ?)'''
    
    connection.execute(query_2, [snp_name, clin_sig, n_publications]) 
    
    connection.commit()
    connection.close()


def get_snp_info(red_id_list):
    '''
    Get info about SNP from soup object and add it to the SQL DB.
    Wait 5 seconds between get requests.
    Return 'DONE' when done.
    '''
    i = 1
    
    for snp_name in red_id_list:
        
        print(i)
        
        #time.sleep(5)
        
        req_url = f'https://www.ncbi.nlm.nih.gov/snp/{snp_name}#clinical_significance'

        resp = requests.get(req_url, params={'horizontal_tab': 'true'})
        soup = BeautifulSoup(resp.content, 'lxml')

        db_build = soup.find('div', {'class': 'accession usa-width-one-third'}).text.split()[2]

        snp_info = soup.find_all('dl', {'class': 'usa-width-one-half'})
        dd_info = snp_info[0].find_all('dd')
        build_chr_pos = dd_info[1].find_all('span') 

        assembly_build = build_chr_pos[1].text.strip()
        position = build_chr_pos[0].text.split(':')[1]
        chromosome = build_chr_pos[0].text.split(':')[0]

        alleles = ''.join(dd_info[2].text.split())
    
        clin_sig = snp_info[1].find('dd').text.strip()
        
        publ_info = snp_info[1].find('a', {'id': 'snp_pub_count'})
        
        if publ_info:
            n_publications = publ_info.text.split()[0]
        else:
            n_publications=0
        
        add_to_sql(snp_name,
                   position,
                   chromosome,
                   alleles,
                   db_build,
                   assembly_build,
                   clin_sig,
                   n_publications)
        i += 1
    
    return 'DONE'


def create_SNP_db():
    '''
    Create DB with two tables with info about human genes SNP.
    Gets information about the name of human genes from the "genomics.senescence.info" site.
    When searching for SNPs for each gene in SNP NCBI, returns no more than 20 SNPs.
    Write found SNPs to the SNP_ids.txt file.
    Search info about each SNP in SNP NCBI.
    Add info about found SNPs to SNP_human.db.
    Return DONE when done.
    
    '''
    
    connection = sqlite3.connect('./data/SNP_human.db')

    connection.execute('''CREATE TABLE IF NOT EXISTS SNP_data ( 
                           SNP_Name TEXT, 
                           Alleles TEXT, 
                           Position TEXT, 
                           Chr TEXT,
                           dbSNP_Build TEXT,
                           Assembly_Build TEXT)''')

    connection.execute('''CREATE TABLE IF NOT EXISTS clin_SNP (
                           SNP_Name TEXT,
                           Clinical_Significance TEXT,
                           N_Publications TEXT,
                           FOREIGN KEY (SNP_Name) REFERENCES SNP_data (SNP_Name))''')

    connection.commit()
    connection.close()
    ''' # Done if file
    genes_list = get_genes()

    id_list = get_id_list(genes_list)
    red_id_list = list(set([j for sub in id_list for j in sub]))

    with open('./data/SNP_ids.txt', 'w') as output_file:
        for rsid in red_id_list:
            output_file.write(rsid + '\n')
    '''
    
    with open('./data/SNP_ids.txt') as f:
        red_id_list = f.read().splitlines()
    
    return get_snp_info(red_id_list)

In [ ]:
create_SNP_db()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
